# Powerlifting

In [47]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly as pt
import plotly.express as px
from geopy.exc import GeocoderTimedOut 
from geopy.geocoders import Nominatim 

In [48]:
powerlifting = pd.read_csv("Powerlifting/openpowerlifting.csv")
powerlifting.head()

,MeetID,Name,Sex,Equipment,Age,Division,BodyweightKg,WeightClassKg,Squat4Kg,BestSquatKg,Bench4Kg,BestBenchKg,Deadlift4Kg,BestDeadliftKg,TotalKg,Place,Wilks
0,0,Angie Belk Terry,F,Wraps,47.0,Mst 45-49,59.60,60,NaN,47.63,NaN,20.41,NaN,70.31,138.35,1,155.05
1,0,Dawn Bogart,F,Single-ply,42.0,Mst 40-44,58.51,60,NaN,142.88,NaN,95.25,NaN,163.29,401.42,1,456.38
2,0,Dawn Bogart,F,Single-ply,42.0,Open Senior,58.51,60,NaN,142.88,NaN,95.25,NaN,163.29,401.42,1,456.38
3,0,Dawn Bogart,F,Raw,42.0,Open Senior,58.51,60,NaN,NaN,NaN,95.25,NaN,NaN,95.25,1,108.29
4,0,Destiny Dula,F,Raw,18.0,Teen 18-19,63.68,67.5,NaN,NaN,NaN,31.75,NaN,90.72,122.47,1,130.47


In [49]:
meets = pd.read_csv("Powerlifting/meets.csv")
meets.head()

,MeetID,MeetPath,Federation,Date,MeetCountry,MeetState,MeetTown,MeetName
0,0,365strong/1601,365Strong,2016-10-29,USA,NC,Charlotte,2016 Junior & Senior National Powerlifting Cha...
1,1,365strong/1602,365Strong,2016-11-19,USA,MO,Ozark,Thanksgiving Powerlifting Classic
2,2,365strong/1603,365Strong,2016-07-09,USA,NC,Charlotte,Charlotte Europa Games
3,3,365strong/1604,365Strong,2016-06-11,USA,SC,Rock Hill,Carolina Cup Push Pull Challenge
4,4,365strong/1605,365Strong,2016-04-10,USA,SC,Rock Hill,Eastern USA Challenge


In [50]:
openpowerlifting = pd.merge(left=powerlifting, right=meets, how='inner', on='MeetID')
openpowerlifting.head()

,MeetID,Name,Sex,Equipment,Age,Division,BodyweightKg,WeightClassKg,Squat4Kg,BestSquatKg,...,TotalKg,Place,Wilks,MeetPath,Federation,Date,MeetCountry,MeetState,MeetTown,MeetName
0,0,Angie Belk Terry,F,Wraps,47.0,Mst 45-49,59.60,60,NaN,47.63,...,138.35,1,155.05,365strong/1601,365Strong,2016-10-29,USA,NC,Charlotte,2016 Junior & Senior National Powerlifting Cha...
1,0,Dawn Bogart,F,Single-ply,42.0,Mst 40-44,58.51,60,NaN,142.88,...,401.42,1,456.38,365strong/1601,365Strong,2016-10-29,USA,NC,Charlotte,2016 Junior & Senior National Powerlifting Cha...
2,0,Dawn Bogart,F,Single-ply,42.0,Open Senior,58.51,60,NaN,142.88,...,401.42,1,456.38,365strong/1601,365Strong,2016-10-29,USA,NC,Charlotte,2016 Junior & Senior National Powerlifting Cha...
3,0,Dawn Bogart,F,Raw,42.0,Open Senior,58.51,60,NaN,NaN,...,95.25,1,108.29,365strong/1601,365Strong,2016-10-29,USA,NC,Charlotte,2016 Junior & Senior National Powerlifting Cha...
4,0,Destiny Dula,F,Raw,18.0,Teen 18-19,63.68,67.5,NaN,NaN,...,122.47,1,130.47,365strong/1601,365Strong,2016-10-29,USA,NC,Charlotte,2016 Junior & Senior National Powerlifting Cha...


In [86]:
count_state=openpowerlifting.groupby(['MeetState']).size()
#print(count_state)
#the full dataframe
df_plot = pd.DataFrame(count_state.sort_values(ascending = False))

#the top 5
df2 = df_plot[:10].copy()

new_row = count_state[:-10].sum()

print(df2.type)
df2 = pd.concat([df2, new_row])
print(df2)

AttributeError: 'DataFrame' object has no attribute 'type'

In [ ]:
'''


#combining top 5 with others
df2 = pd.concat([df2, new_row])
# Creating plot
fig = plt.figure(figsize =(10, 10))
plt.pie(df2, labels=df2.index, autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2)
 
# show plot
plt.show()
'''

In [51]:
import plotly.figure_factory as ff

In [88]:
px.choropleth_mapbox(openpowerlifting,
                     locations='MeetTown',
                     color='MeetID',
                     hover_name='MeetTown',
                     hover_data=['MeetID'],
                     #scope='america',
                     mapbox_style='carto-positron',center={'lat': 24, 'lon':78},
                     zoom=3)



TypeError: choropleth_mapbox() got an unexpected keyword argument 'scope'